In [34]:
from recbole.quick_start.quick_start import load_data_and_model
from recbole.utils.case_study import full_sort_scores, full_sort_topk

### Reload model

In [2]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='./saved/GRU4Rec-Jun-14-2024_12-44-49.pth',
)

16 Jun 18:17    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ./data/Food
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 20
train_batch_size = 512
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 512
metric_decimal_place = 4

Dataset Hyper Parameters:
fie

### Convert external user id into internal user id

In [27]:
dataset.field2token_id

{'user_id': {'[PAD]': 0,
  '38094': 1,
  '1293707': 2,
  '8937': 3,
  '126440': 4,
  '57222': 5,
  '52282': 6,
  '124416': 7,
  '2000192946': 8,
  '76535': 9,
  '273745': 10,
  '353911': 11,
  '190375': 12,
  '468945': 13,
  '255338': 14,
  '1171894': 15,
  '136726': 16,
  '68960': 17,
  '618928': 18,
  '217118': 19,
  '2000049093': 20,
  '202555': 21,
  '353579': 22,
  '635209': 23,
  '681408': 24,
  '684460': 25,
  '900992': 26,
  '135017': 27,
  '224088': 28,
  '582223': 29,
  '1413963': 30,
  '483827': 31,
  '56680': 32,
  '5060': 33,
  '935485': 34,
  '539686': 35,
  '6258': 36,
  '102602': 37,
  '172467': 38,
  '2042100': 39,
  '2002068522': 40,
  '58332': 41,
  '160497': 42,
  '67026': 43,
  '183565': 44,
  '101823': 45,
  '349752': 46,
  '446143': 47,
  '226989': 48,
  '868654': 49,
  '1143059': 50,
  '302867': 51,
  '1316788': 52,
  '1447190': 53,
  '1003295': 54,
  '930021': 55,
  '241697': 56,
  '980899': 57,
  '158966': 58,
  '827374': 59,
  '1719774': 60,
  '49304': 61,
  

In [28]:
external_user_ids = ['38094', '1293707']
uid_series = dataset.token2id(dataset.uid_field, external_user_ids)

In [29]:
uid_series

array([1, 2])

### Get scores of every user-item pairs

In [31]:
score = full_sort_scores(uid_series, model, test_data, device=config['device'])
score  # score of all items

tensor([[   -inf,  0.0234, -0.7613,  ..., -1.7450, -6.3485, -2.8508],
        [   -inf, -2.8919, -1.8695,  ..., -4.0996, -7.1146, -2.7041]],
       device='cuda:0')

In [35]:
score[0, dataset.token2id(dataset.iid_field, ['242', '302'])] # score of item ['242', '302'] for user 1.

tensor([-0.3603, -1.9295], device='cuda:0')

Note that the score of `[pad]` and history items (for non-repeatable recommendation) will be set into `-inf`.

### Get the top ranked item for each user

In [36]:
topk_score, topk_iid_list = full_sort_topk(uid_series, model, test_data, k=10, device=config['device'])
topk_score  # scores of top 10 items

tensor([[4.8438, 4.5941, 4.4091, 4.0218, 4.0056, 3.8765, 3.8044, 3.7664, 3.7445,
         3.7026],
        [2.1452, 1.9899, 1.9859, 1.9354, 1.9292, 1.9248, 1.9182, 1.9169, 1.8920,
         1.8614]], device='cuda:0')

In [37]:
topk_iid_list  # internal id of top 10 items

tensor([[ 69322, 228679, 113605,  79258, 147315, 159510, 190972, 129281, 107208,
          36049],
        [  9001,  48239, 228532, 100382, 170923, 174672, 155682,  34831,  29983,
          72091]], device='cuda:0')

In [38]:
external_item_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu())
external_item_list  # external tokens of top 10 items

array([['10804', '23687', '31020', '26257', '49200', '18762', '24850',
        '27208', '46773', '22682'],
       ['70646', '150863', '294620', '71373', '32204', '89204', '37455',
        '115153', '51537', '67256']], dtype='<U6')